# Structure-Based Screening Pipeline

Import libraries

Note: Make sure you have biotide installed:
`conda install -c conda-forge biotite`

In [ ]:
from pathlib import Path 

import biotite.database.rcsb as rcsb
import nglview as nv

from openbabel import pybel

## Task 1: Get protein structure from PDB

Generally, protein structures can be downloaded from the protein data bank ([PDB](http://www.rcsb.org)), the largest freely available deposition of 3D protein structures. To get a suitable structure, you can either get it programmatically or manually. 

* For an automated way, we will have a look at the teachopenCADD talktorial [T008:Protein data acquisition](https://projects.volkamerlab.org/teachopencadd/talktorials/T008_query_pdb.html).
* To make it easier for you, you can also choose a structure manually. 
    * On the website, you can use the upper right search field and search for `EGFR` or `P00533`
    * A list of matching entries will be given. 

Your selection of the PDB structure to work with may depend on diverse criteria. To refine your search you can for example restrict the results to
* Human only
* X-ray structure
* Having a good resolution (< 2Å)
* and a recent deposition date (>2010) 

For kinases, more and detailed information is available on the [**KLIFS**](https://klifs.net/) webpage. Here, you also find additional information about the `DFG-loop` state of the `available structures` as well as some `quality criteria`, such as the number of missing residues.

### 1.1. Identify query structure(s) from PDB 

* Define selection criteria

In [4]:
uniprot_id = "P00533"
after_deposition_date = "2010-01-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 2.0
n_chains = 1
min_ligand_molecular_weight = 100.0

* Set-up the `rcsb` query

In [7]:
# Uniprot id
query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
# Deposition date
query_by_deposition_date = rcsb.FieldQuery(
    "rcsb_accession_info.deposit_date", greater=after_deposition_date
)
# X-ray
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
# Resolution
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
# Chain
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
# Ligand size
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

* Combine the query

In [8]:
query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_deposition_date,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)

* Perform the query 

Note: this step can take some time depending on the connection

In [10]:
pdb_ids = rcsb.search(query)

In [12]:
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(pdb_ids)

Number of matches: 21
Selected PDB IDs:
['3POZ', '3W2S', '3W32', '3W33', '4I22', '4WKQ', '4WRG', '5GNK', '5HG5', '5HG7', '5HG8', '5U8L', '5UG8', '5UG9', '5UGA', '5UGC', '7SI1', '8A27', '8A2A', '8A2B', '8A2D']


### 1.2. Retrieve structure from the Protein Data Bank

Your task is to choose a suitable EGFR structure for docking. Make sure that your structure is complete (no missing residues) around the active site. 
* To keep it simple, try to select a structure containing only one biological unit. 
* Note down the PDB ID of your structure. 
* Using the example code below, you can retrieve the structure from PDB using the ID.

In [1]:
# Your pdb entry
pdb_id = "2ito"

In [2]:
structure = Structure.from_pdbid(pdb_id)
# element information maybe missing, but important for subsequent PDBQT conversion
if not hasattr(structure.atoms, "elements"):
    structure.add_TopologyAttr("elements", structure.atoms.types)
structure

NameError: name 'Structure' is not defined

## Task 2: Binding site detection

If a co-crystallized ligand is not present in your protein structure, you need to specify the binding site manually. Several pocket detection algorithms were developed to detect protein pockets suitable for binding of drug-like molecules. 

Helpful talktorial: [T014-Binding site detection](https://projects.volkamerlab.org/teachopencadd/talktorials/T014_binding_site_detection.html)


### Task 2.1
Use the DoGSiteScorer available at [proteins.plus](http://www.proteins.plus) to identify pockets in your protein-ligand complex. Check the “subpockets” box to receive more fine-grained pocket results. Is the binding site of the co-crystallized ligand scored the best (is it considered druggable)? Is the predicted binding site covering the co-crystallized ligand well? 


### Task 2.2
[*optional*] Download the results from DoGSiteScorer. In the zipped archive locate the PDB file of your favorite predicted binding site inside the residues directory. Open the PDB file in a text editor and note down the geometric center and radius of the binding site. You can visualize the identified geometric center and radius from DoGSiteScorer in PyMol with the use of the pseudoatom command. After creating the pseudoatom with the right coordinates, you can display it as a sphere and change the radius accordingly (Tip: Use the sphere_transparency setting to better visualize all components together). Are you confident about the selected parameters or would you adjust them?

### Task 2.3
[*optional*] Similar to proteins.plus, PyMol can also be used to visualize volumetric maps. In the downloaded DoGSiteScorer results locate your preferred binding pocket in CCP4 format and load it together with the protein-ligand complex into PyMol.




## Task 3: Docking

Before we can start with the docking, we need to prepare our structures. 

### Task 3.1
To automatically prepare the structure, use Protoss (proteins.plus). Protoss adds missing hydrogen atoms to protein structures (PDB-format) and detects reasonable protonation states, tautomers, and hydrogen coordinates of both protein and ligand molecules. Upload your chosen structure to the server and store the optimized PDB structure.

### Task 3.2

For the docking calculations, we need to separate the ligand and protein. Thus, the ligand must be extracted from the protein structure and deleted from the protein structure for docking calculations, otherwise there is no space in the binding site.

**TODO**

### Task 3.3 
Programs based on the AutoDock software require protein and ligand to be prepared in PDBQT format (AutoDock FAQ). This file format is very similar to the PDB format but additionally stores information about atom types and partial charges. Luckily, the OpenBabel package provides functionality for converting between different file formats and calculating partial charges (command: `obabel protein.pdb -O protein.pdbqt --partialcharge gasteiger`). Check if the generated PDBQT files for protein and ligand contain information about the assigned charges for each atom. In theory, we could also use OpenBabel to add hydrogens. However, Protoss already added missing hydrogens considering protein and ligand in complex, which is more accurate than protonating protein and ligand separately.

### Task 3.4
For docking your filtered compounds, you need the 3D structures of the molecules. Calculate the 3D structure of each molecule and save them in a single SDF file.

Tip:
```ruby
w = Chem.SDWriter('data/foo.sdf')
for m in mols:
    AllChem.EmbedMolecule(m)		# calculates 3D structure
    AllChem.UFFOptimizeMolecule(m)	# improves quality of the conformation
    w.write(m)
```

In [ ]:
# calculate 3D structure and write to SDF file

### Task 3.5

With everything in hand, we can finally run our docking calculation using Smina with the following command:
```
smina --ligand test_ligand.sdf --receptor protein.pdbqt --out docking_poses.sdf --autobox_ligand ligand.pdbqt --num_modes 1
````
We will use the binding site of the co-crystallized ligand as a template for docking the compounds. But if you have time, you could also use the geometric center and radius from DogSiteScorer as the binding pocket. Your docked compounds will all be in `docking_poses.sdf`. 

## Task 4: Visualize docking results 

To visualize your docking results, we will use the NGLView package which can display the structures in 3D and interactively in jupyter directly. 

Helpful tutorial: [T015-Protein ligand docking](https://projects.volkamerlab.org/teachopencadd/talktorials/T015_protein_ligand_docking.html)


### Task 4.1 

First you have to split your SD file with the docking poses, so that each docked compound is in a separate file. This makes it easier to visualize with NGLView in the next task. Call the function defined below to split your docking pose file. 

In [ ]:
def split_sdf_file(sdf_path):
    """
    Split an SDF file into seperate files for each molecule.
    Each file is named with consecutive numbers.

    Parameters
    ----------
    sdf_path: str or pathlib.Path
        Path to SDF file that should be split.
    """
    sdf_path = Path(sdf_path)
    stem = sdf_path.stem
    parent = sdf_path.parent
    molecules = pybel.readfile("sdf", str(sdf_path))
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)
    return


In [ ]:
# split sd file 

### Task 4.2 

Now you can use NGLView to inspect your docking results one by one.
- One simple example to use NGLView given a PDB ID: 

In [ ]:
view = nv.show_pdbid(pdb_id)
view


To view your docking pose, you will need to display the ligand from your sdf and the protein: 

```ruby
docking_pose_id = 1
view = nv.show_structure_file(
    str(DATA / f"docking_poses_{docking_pose_id}.sdf"),         # add path to your docking pose 
    representations=[{"params": {}, "type": "licorice"}],
)
view.add_pdbid(pdb_id)
view
``` 

In [ ]:
# use nglview to visualize docking pose  

To interpret your docking results, there are several indicators you can look at if you have time:  
- Which compounds/poses have the best docking scores (given as output from smina directly)?
- Visually inspect the results using a molecule viewer to analyze the binding mode and compare them to the co-crystallized ligand or other known EGFR inhibitors. 
- Is the binding site well covered? Do protein and pose fit to each other (shape and physico-chemically). Are key interactions (e.g. hinge binding motif) present. You can also use [PoseView](https://proteins.plus/) for inspection which generates 2D diagram of protein-ligand interactions.

Based on these (and other criteria you can think of), you can filter your compounds down further. In the end, you should select a few compounds that seem to be most promising to you.